# INFO

This notebook will be used for construction and testing purposes while designing model within Kedro framework. 

In [1]:
##############################################################################
# It is recommended to create new virtual environment for each Kedro project #
##############################################################################

# Uncomment and run the line below if your environment does't have
# Kedro or any other dependencies needed.

#! pip install -r requirements.txt
%load_ext kedro.ipython

[11/05/24 19:18:17] INFO     Using                                                                  ]8;id=380682;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=297712;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/k                
                             edro/framework/project/rich_logging.yml' as logging configuration.                    

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=463005;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=681348;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=606584;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=939456;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=374681;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=929716;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/andrejcemeris/Projects/Python3/lending-club.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/05/24 19:18:20] INFO     Kedro project lending_club                                             ]8;id=106242;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=879730;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=830685;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=91765;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[11/05/24 19:18:23] INFO     Registered line magic 'run_viz'                                        ]8;id=9562;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=488838;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\


----

# Baseline model
TODO Remove
As the dataset needs transformation like imputation and normalization, for avoiding data leakage, all transformations will be done within model pipeline and fitting only on training data on model fitting stage. So I'm going to split initial typed dataset to train/test sets and balance train set, ignoring all previously transformed datasets.

In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import pandas as pd
import logging
from lending_club.pipelines.analysis.nodes import features_eng
from lending_club.pipelines.encode.nodes import _default_status
from imblearn.pipeline import make_pipeline as imb_make_pipeline

logger = logging.getLogger(__name__)

def split_dataset(df: pd.DataFrame, df_fe: pd.DataFrame, params: dict):
    y = _default_status(df, params)
    X = pd.concat([df, df_fe], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=params['test_size'],
        random_state=params['random_state']
    )
    return X_train, X_test, y_train, y_test

def train_model(X_train, y_train, regressor, params: dict):
    try:
        regressor.set_params(**params['fit_options']).fit(X_train, y_train)
    except:
        regressor.fit(X_train, y_train)
    return regressor

def evaluate_metrics(model: object, params: dict, X_true: object, y_true: object) -> pd.DataFrame:
    y_pred = model.predict(X_true)
    # NN model return probabilities that we should translate to
    # True/False based on treshold = 0.5
    if not isinstance(y_pred, list):
        y_pred = (y_pred > 0.5)
    metrics = pd.DataFrame()
    metrics['accuracy']  = {params['name']: accuracy_score(y_true, y_pred)}
    metrics['precision'] = {params['name']: precision_score(y_true, y_pred)}
    metrics['recall']    = {params['name']: recall_score(y_true, y_pred)}
    metrics['f1']        = {params['name']: f1_score(y_true, y_pred)}
    metrics['roc_auc']   = {params['name']: roc_auc_score(y_true, y_pred)}
    metrics['conf_mtx']  = {params['name']: confusion_matrix(y_true, y_pred)}
    print("Model's scores:\n", metrics)

def model_pipeline(params: dict):

    # split important features to assign preprocessing steps
    category_feat = [f for f in (params['category'] + [params['emp_len']]) if f in params['model_features']]
    numeric_feat_zero = [f for f in params['fill_zero'] if f in params['model_features']]
    numeric_feat_med = [f for f in params['fill_med'] if f in params['model_features']]

    # transformer to replace missing numeric values by 0
    numeric_feat_zero_transformer = make_pipeline(
        SimpleImputer(strategy='constant', fill_value=0),
        StandardScaler()
    )
    # transformer to replace missing numeric values by median
    numeric_feat_med_transformer = make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler()
    )
    # assemble transformers to preprocessing pipe
    preprocessing = make_column_transformer(
        (OrdinalEncoder(), category_feat),
        (numeric_feat_zero_transformer, numeric_feat_zero),
        (numeric_feat_med_transformer, numeric_feat_med)
    )

    regressor_params = params['model_options']['regressor_options']

    pipeline = imb_make_pipeline(
        preprocessing,
        SMOTE(random_state=params['random_state']),
        # CatBoostClassifier(
        #     iterations=regressor_params['iterations'],						# Maximum number of boosting iterations
        #     learning_rate=regressor_params['learning_rate'],					# Learning rate
        #     eval_metric=regressor_params['eval_metric'],					    # Metric to monitor
        #     custom_loss=regressor_params['custom_loss'],                      # Additional metrics to plot
        #     early_stopping_rounds=regressor_params['early_stopping_rounds'],	# Stop if no improvement after X iterations
        #     od_type=regressor_params['od_type'],						        # Overfitting detection type (detect after fixed number of non-improving iterations)
        #     random_seed=regressor_params['random_seed'],
        #     verbose=regressor_params['verbose'],						        # Print log every X iterations
        #     eval_fraction=regressor_params['eval_fraction']                   # Fraction of training dataset for validation
        # )
        RandomForestClassifier(**regressor_params)
    )
    return pipeline
%reload_kedro
# Load parameters
params = catalog.load("parameters")
params['model_features'] = catalog.load("params:model_features")
params['model_options'] = catalog.load("params:model_options")

# Construct pipeline
model = model_pipeline(params)
model

[11/06/24 00:09:49] INFO     Resolved project path as:                                              ]8;id=533319;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=751079;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/andrejcemeris/Projects/Python3/lending-club.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/06/24 00:09:51] INFO     Kedro project lending_club                                             ]8;id=69401;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=910713;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=887093;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=397763;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[11/06/24 00:09:52] INFO     Registered line magic 'run_viz'                                        ]8;id=277539;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=603212;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\

                    INFO     Loading data from parameters (MemoryDataset)...                    ]8;id=922621;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=425034;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from params:model_features (MemoryDataset)...         ]8;id=788014;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=348005;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from params:model_options (MemoryDataset)...          ]8;id=483657;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=736651;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['home_ownership',
                                                   'sub_grade', 'purpose',
                                                   'verification_status',
                                                   'application_type',
                                                   'verification_status_joint']),
                                                 ('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  [...
                                                   'avg_cur_bal', 'tot_cur_bal',
                                                   'num_actv_bc_tl',
                                                   'num_actv_rev_tl',
                                                   'num_op_rev_tl',
                                                   'mo_sin_rcnt_rev_tl_op',
                                                   'num_tl_op_past_12m',
                                                   'total_il_high_credit_limit',
                                                   'mo_sin_rcnt_tl',
                                                   'mo_sin_old_rev_tl_op',
                                                   'total_bc_limit',
                                                   'acc_open_past_24mths', ...])])),
                ('smote', SMOTE(random_state=42)),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', random_state=42))])

In [88]:
df = catalog.load("intermediate_lc_dataset")
X_train, X_test, y_train, y_test = split_dataset(df, features_eng(df, params), params)

[11/06/24 00:09:53] INFO     Loading data from intermediate_lc_dataset (ParquetDataset)...      ]8;id=708899;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=730195;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [89]:
fitted = model.fit(X_train, y_train)
fitted

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['home_ownership',
                                                   'sub_grade', 'purpose',
                                                   'verification_status',
                                                   'application_type',
                                                   'verification_status_joint']),
                                                 ('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  [...
                                                   'avg_cur_bal', 'tot_cur_bal',
                                                   'num_actv_bc_tl',
                                                   'num_actv_rev_tl',
                                                   'num_op_rev_tl',
                                                   'mo_sin_rcnt_rev_tl_op',
                                                   'num_tl_op_past_12m',
                                                   'total_il_high_credit_limit',
                                                   'mo_sin_rcnt_tl',
                                                   'mo_sin_old_rev_tl_op',
                                                   'total_bc_limit',
                                                   'acc_open_past_24mths', ...])])),
                ('smote', SMOTE(random_state=42)),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', random_state=42))])

In [90]:
evaluate_metrics(model, params['model_options'], X_test, y_test)

Model's scores:
                         accuracy  precision    recall        f1   roc_auc  \
randomforestclassifier    0.8722    0.37037  0.065466  0.111266  0.524987   

                                           conf_mtx  
randomforestclassifier  [[17284, 272], [2284, 160]]  
